In [1]:
import pandas as pd
import numpy as np
import collections
import gensim 
from gensim.models import word2vec, phrases
from gensim.parsing.preprocessing import remove_stopwords, strip_punctuation, strip_numeric,\
                    strip_non_alphanum, strip_multiple_whitespaces, strip_short
from textblob import TextBlob, Word

import re
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
%matplotlib inline


from strsimpy.cosine import Cosine

C:\Users\wen\Anaconda3\lib\site-packages\requests\__init__.py:80: RequestsDependencyWarning: urllib3 (1.25.10) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [11]:
recipe = pd.read_csv('raw-data_recipe.csv')
recipe = recipe.drop(columns=['image_url', 'cooking_directions', 'reviews'])
recipe.head()

,recipe_id,recipe_name,aver_rate,review_nums,ingredients,nutritions
0,222388,Homemade Bacon,5.000000,3,pork belly^smoked paprika^kosher salt^ground b...,"{u'niacin': {u'hasCompleteData': False, u'name..."
1,240488,"Pork Loin, Apples, and Sauerkraut",4.764706,29,sauerkraut drained^Granny Smith apples sliced^...,"{u'niacin': {u'hasCompleteData': False, u'name..."
2,218939,Foolproof Rosemary Chicken Wings,4.571429,12,chicken wings^sprigs rosemary^head garlic^oliv...,"{u'niacin': {u'hasCompleteData': True, u'name'..."
3,87211,Chicken Pesto Paninis,4.625000,163,focaccia bread quartered^prepared basil pesto^...,"{u'niacin': {u'hasCompleteData': True, u'name'..."
4,245714,Potato Bacon Pizza,4.500000,2,red potatoes^strips bacon^Sauce:^heavy whippin...,"{u'niacin': {u'hasCompleteData': True, u'name'..."


In [19]:
df = recipe
df.replace("[^a-zA-Z]"," ",regex=True, inplace=True)

In [20]:
df['details'] = df['recipe_name'].str.lower() + df['ingredients'].str.lower()

In [21]:
df.head()

,recipe_id,recipe_name,aver_rate,review_nums,ingredients,nutritions,details
0,222388,Homemade Bacon,5.000000,3,pork belly smoked paprika kosher salt ground b...,u niacin u hasCompleteData False u name...,homemade baconpork belly smoked paprika kosher...
1,240488,Pork Loin Apples and Sauerkraut,4.764706,29,sauerkraut drained Granny Smith apples sliced ...,u niacin u hasCompleteData False u name...,pork loin apples and sauerkrautsauerkraut dr...
2,218939,Foolproof Rosemary Chicken Wings,4.571429,12,chicken wings sprigs rosemary head garlic oliv...,u niacin u hasCompleteData True u name ...,foolproof rosemary chicken wingschicken wings ...
3,87211,Chicken Pesto Paninis,4.625000,163,focaccia bread quartered prepared basil pesto ...,u niacin u hasCompleteData True u name ...,chicken pesto paninisfocaccia bread quartered ...
4,245714,Potato Bacon Pizza,4.500000,2,red potatoes strips bacon Sauce heavy whippin...,u niacin u hasCompleteData True u name ...,potato bacon pizzared potatoes strips bacon sa...


In [23]:
details=[]
for row in range(0,len(df.index)):
    details.append(df.iloc[row,6])

In [24]:
details

['homemade baconpork belly smoked paprika kosher salt ground black pepper',
 'pork loin  apples  and sauerkrautsauerkraut drained granny smith apples sliced large onion caraway seeds apple cider divided brown sugar rub  thai seasoning salt garlic powder ground black pepper boneless pork loin roast',
 'foolproof rosemary chicken wingschicken wings sprigs rosemary head garlic olive oil lemon pepper seasoned salt',
 'chicken pesto paninisfocaccia bread quartered prepared basil pesto diced cooked chicken diced green bell pepper diced red onion shredded monterey jack cheese',
 'potato bacon pizzared potatoes strips bacon sauce  heavy whipping cream butter minced garlic grated parmesan cheese crust  warm water      degrees f    degrees c  honey active dry yeast vegetable oil all purpose flour shredded mozzarella cheese',
 'latin inspired spicy cream chicken stewskinless boneless chicken breast halves diced tomatoes green salsa black beans rinsed and drained pinto beans drained and rinsed ker

In [49]:
# input json  
#{
#    "uuid": 1234,
#    "user_id": 1234,
#    "request_id": 1,
#    "request_time": "9/28/2020",
#    "user input": {
#        "motherhood_stage": "pregnancy first trimester",
#        "ingredient_list": ["chicken", "tomato"],
#        "nutrition_focus": ["full-eval","iron"],
#        "if_vegetierian": true,
#        "ingredient_exclusion": ["onion","cilantro","cheese"]
#    }
#}

In [68]:
df['input'] = "chicken, tomato"

In [69]:
df.head()

,recipe_id,recipe_name,aver_rate,review_nums,ingredients,nutritions,details,input,p0,p1,cosine_sim
0,222388,Homemade Bacon,5.000000,3,pork belly smoked paprika kosher salt ground b...,u niacin u hasCompleteData False u name...,homemade baconpork belly smoked paprika kosher...,"chicken, tomato","{'ch': 1, 'hi': 1, 'ic': 1, 'ck': 1, 'ke': 1, ...","{'ho': 1, 'om': 1, 'me': 1, 'em': 1, 'ma': 1, ...",0.130437
1,240488,Pork Loin Apples and Sauerkraut,4.764706,29,sauerkraut drained Granny Smith apples sliced ...,u niacin u hasCompleteData False u name...,pork loin apples and sauerkrautsauerkraut dr...,"chicken, tomato","{'ch': 1, 'hi': 1, 'ic': 1, 'ck': 1, 'ke': 1, ...","{'po': 3, 'or': 2, 'rk': 4, 'k ': 3, ' l': 3, ...",0.072778
2,218939,Foolproof Rosemary Chicken Wings,4.571429,12,chicken wings sprigs rosemary head garlic oliv...,u niacin u hasCompleteData True u name ...,foolproof rosemary chicken wingschicken wings ...,"chicken, tomato","{'ch': 1, 'hi': 1, 'ic': 1, 'ck': 1, 'ke': 1, ...","{'fo': 1, 'oo': 2, 'ol': 2, 'lp': 1, 'pr': 2, ...",0.197546
3,87211,Chicken Pesto Paninis,4.625000,163,focaccia bread quartered prepared basil pesto ...,u niacin u hasCompleteData True u name ...,chicken pesto paninisfocaccia bread quartered ...,"chicken, tomato","{'ch': 1, 'hi': 1, 'ic': 1, 'ck': 1, 'ke': 1, ...","{'ch': 3, 'hi': 2, 'ic': 5, 'ck': 3, 'ke': 3, ...",0.232565
4,245714,Potato Bacon Pizza,4.500000,2,red potatoes strips bacon Sauce heavy whippin...,u niacin u hasCompleteData True u name ...,potato bacon pizzared potatoes strips bacon sa...,"chicken, tomato","{'ch': 1, 'hi': 1, 'ic': 1, 'ck': 1, 'ke': 1, ...","{'po': 3, 'ot': 2, 'ta': 3, 'at': 4, 'to': 2, ...",0.216146


In [71]:
cosine = Cosine(2)
df["p0"] = df["input"].apply(lambda s: cosine.get_profile(s)) 
df["p1"] = df["details"].apply(lambda s: cosine.get_profile(s)) 
df["cosine_sim"] = [cosine.similarity_profiles(p0,p1) for p0,p1 in zip(df["p0"],df["p1"])]

df_2 = df.drop(["p0", "p1"], axis=1)

In [76]:
output = df_2.nlargest(5, 'cosine_sim')

In [77]:
output

,recipe_id,recipe_name,aver_rate,review_nums,ingredients,nutritions,details,input,cosine_sim
27704,14093,Sun dried Tomatoes II,5.000000,6,cherry tomatoes,u niacin u hasCompleteData True u name ...,sun dried tomatoes iicherry tomatoes,"chicken, tomato",0.585850
47357,17900,Tomato Florentine Soup II,3.875000,13,chicken stock tomato sauce tomato juice tomato...,u niacin u hasCompleteData False u name...,tomato florentine soup iichicken stock tomato ...,"chicken, tomato",0.558556
4817,245792,Chicken Tomato and Spinach Quesadillas,4.428571,7,flour tortillas,u niacin u hasCompleteData True u name ...,chicken tomato and spinach quesadillasflour ...,"chicken, tomato",0.537853
30149,14081,Sun dried Tomatoes I,4.728814,93,tomatoes salt to taste,u niacin u hasCompleteData True u name ...,sun dried tomatoes itomatoes salt to taste,"chicken, tomato",0.534417
46202,18062,Mock Chicken,4.295455,37,firm tofu,u niacin u hasCompleteData True u name ...,mock chickenfirm tofu,"chicken, tomato",0.533002


In [78]:
result = output['recipe_id']

In [75]:
result

27704     14093
47357     17900
4817     245792
30149     14081
46202     18062
45921    147360
10423     15737
11451    192533
48987     13091
46802     21083
Name: recipe_id, dtype: int64